In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('c:/Users/HP/Desktop/mnist/',one_hot=True)

Extracting c:/Users/HP/Desktop/mnist/train-images-idx3-ubyte.gz
Extracting c:/Users/HP/Desktop/mnist/train-labels-idx1-ubyte.gz
Extracting c:/Users/HP/Desktop/mnist/t10k-images-idx3-ubyte.gz
Extracting c:/Users/HP/Desktop/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
print('train',len(data.train.labels))
print('test',len(data.test.labels))
print('validation',len(data.validation.labels))

train 55000
test 10000
validation 5000


In [4]:
x = tf.placeholder(tf.float32,[None,784])
y_true = tf.placeholder(tf.float32,([None,10]))
y_true_cls = tf.arg_max(y_true,dimension=1)

The convolutional layers expect `x` to be encoded as a 4-dim tensor so we have to reshape it so its shape is instead `[num_images, img_height, img_width, num_channels]`. Note that `img_height == img_width == img_size` and `num_images` can be inferred automatically by using -1 for the size of the first dimension. So the reshape operation is:

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [6]:
w = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
number_channels = 1 #1 for gray scale

In [7]:
logits = tf.matmul(x,w)+b
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [8]:
# convolution layer #1
filter_size1 = 5 #5x5 pixels
num_filters1 = 16 #there are 16 of these types

# convolution layer #2
filter_size2 = 5 #5x5 pixels
num_filters2 = 36 #there are 36 of these types

#fully connected layer
fc_size = 128 #num of neurons in fc layer

In [9]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev = 0.05))
def new_biases(length):
    return tf.Variable(tf.constant(0.05,shape=[length]))

In [10]:
#pooling 2x2 max pooling
def new_convolution_layer(input,num_of_input_channels,filter_size,num_of_filters,use_pooling = True):
        # Shape of the filter-weights for the convolution.
        shape = [filter_size,filter_size,num_of_input_channels,num_of_filters]
        # Create new weights aka. filters with the given shape.
        weights = new_weights(shape=shape)
        # Create new biases, one for each filter.
        biases = new_biases(length=num_of_filters)
        # Create the TensorFlow operation for convolution.
        # Note the strides are set to 1 in all dimensions.
        # The first and last stride must always be 1,
        # because the first is for the image-number and
        # the last is for the input-channel.
        # But e.g. strides=[1, 2, 2, 1] would mean that the filter
        # is moved 2 pixels across the x- and y-axis of the image.
        # The padding is set to 'SAME' which means the input image
        # is padded with zeroes so the size of the output is the same.
        layer = tf.nn.conv2d(input=input,
                             filter=weights,
                             strides=[1, 1, 1, 1],
                             padding='SAME')
        # A bias-value is added to each filter-channel.
        layer += biases
    # Use pooling to down-sample the image resolution?
        if use_pooling:
            # This is 2x2 max-pooling, which means that we
            # consider 2x2 windows and select the largest value
            # in each window. Then we move 2 pixels to the next window.
            layer = tf.nn.max_pool(value=layer,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME')

        layer = tf.nn.relu(layer) #RELU
        return layer, weights
        
    

In [11]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()
    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])
    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features
    

In [12]:
def new_fc_layer(input,
                 num_of_inputs,
                 num_of_outputs,
                 use_relu = True):
    
    weights = new_weights(shape=[num_of_inputs,num_of_outputs])
    biases = new_biases(length=num_of_outputs)
    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases
    #use relu
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer
    
    

In [13]:
layer_conv1,weights_conv1 = new_convolution_layer(input = x_image,
                                                  num_of_filters=num_filters1,
                                                  use_pooling=True,
                                                  filter_size=filter_size1,
                                                  num_of_input_channels=number_channels)

In [14]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 16) dtype=float32>

In [15]:
layer_conv2,weights_conv2 = new_convolution_layer(input = layer_conv1,
                                                  num_of_filters=num_filters2,
                                                  use_pooling=True,
                                                  filter_size=filter_size2,
                                                  num_of_input_channels=num_filters1)

In [16]:
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 7, 7, 36) dtype=float32>

In [17]:
layer_flat,num_features = flatten_layer(layer_conv2)

In [18]:
layer_flat

<tf.Tensor 'Reshape_4:0' shape=(?, 1764) dtype=float32>

In [19]:
num_features

1764

In [20]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_of_inputs = num_features,
                         num_of_outputs = 128,
                         use_relu = True)


In [21]:
layer_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 128) dtype=float32>

In [22]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_of_inputs = 128,
                         num_of_outputs = 10,
                         use_relu = False)


In [23]:
layer_fc2

<tf.Tensor 'add_4:0' shape=(?, 10) dtype=float32>

In [24]:
y_pred = tf.nn.softmax(layer_fc2)

In [25]:
y_pred_cls = tf.arg_max(y_pred,dimension=1)

In [26]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [27]:
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [28]:
correct_pred = tf.equal(y_pred_cls,y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [29]:
sess = tf.Session()

In [30]:
sess.run(tf.global_variables_initializer())

In [31]:
train_batch_size = 64

In [32]:
total_iterations = 0
def optimize(num_of_iterations):
    global total_iterations
    for i in range(total_iterations,total_iterations+num_of_iterations):
        x_batch,y_true_batch = data.train.next_batch(train_batch_size)
        feed_dict_train = {x:x_batch,y_true:y_true_batch}
        sess.run(optimizer,feed_dict=feed_dict_train)
        if i%100 == 0 :
            acc = sess.run(accuracy,feed_dict=feed_dict_train)
            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # Print it.
            print(msg.format(i + 1, acc))
    # Update the total number of iterations performed.
    total_iterations += num_of_iterations


    

In [33]:
data.test.cls = np.argmax(data.test.labels, axis=1)

In [34]:
# Split the test-set into smaller batches of this size.
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # Number of images in the test-set.
    num_test = len(data.test.images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = data.test.images[i:j, :]

        # Get the associated labels.
        labels = data.test.labels[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = data.test.cls
    

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [35]:
print_test_accuracy()

Accuracy on Test-Set: 9.9% (992 / 10000)


In [36]:
optimize(num_of_iterations=1)

Optimization Iteration:      1, Training Accuracy:  10.9%


In [37]:
print_test_accuracy()

Accuracy on Test-Set: 9.8% (984 / 10000)


In [38]:
optimize(num_of_iterations=10)

In [39]:
print_test_accuracy()

Accuracy on Test-Set: 12.0% (1199 / 10000)


In [40]:
optimize(num_of_iterations=100)

Optimization Iteration:    101, Training Accuracy:  35.9%


In [41]:
print_test_accuracy()

Accuracy on Test-Set: 42.1% (4208 / 10000)


In [42]:
optimize(num_of_iterations=1000)

Optimization Iteration:    201, Training Accuracy:  59.4%
Optimization Iteration:    301, Training Accuracy:  82.8%
Optimization Iteration:    401, Training Accuracy:  84.4%
Optimization Iteration:    501, Training Accuracy:  87.5%
Optimization Iteration:    601, Training Accuracy:  87.5%
Optimization Iteration:    701, Training Accuracy:  96.9%
Optimization Iteration:    801, Training Accuracy:  89.1%
Optimization Iteration:    901, Training Accuracy:  92.2%
Optimization Iteration:   1001, Training Accuracy:  98.4%
Optimization Iteration:   1101, Training Accuracy: 100.0%


In [43]:
print_test_accuracy()

Accuracy on Test-Set: 92.2% (9224 / 10000)
